In [37]:
using CSV, DataFrames, Dates
# using Tar, CodecZlib

In [38]:
startDate = ["-01-01", "-02-01", "-03-02", "-04-01", "-05-01", "-06-01", "-07-01", "-08-01", "-09-01", "-10-01", "-11-01", "-12-01"]
# account for february ! start from 02-01 to 03-01
endDate = ["-01-31", "-03-01", "-03-31", "-04-30", "-05-31", "-06-30", "-07-31", "-08-31", "-09-30", "-10-31", "-11-30", "-12-31"];

In [40]:
year = 1985
month = 1
url = "http://webservices.ingv.it/fdsnws/event/1/query?starttime=" * string(year) * startDate[month] * "T00%3A00%3A00&endtime=" * string(year) * endDate[month] * "T23%3A59%3A59&minversion=100&orderby=time-asc&timezone=UTC&format=text&limit=15000"
# put directly into dataframe. Note the delimiter:"|" and forcing the String to String
dataset = CSV.read(download(url), DataFrame, delim="|",stringtype=String)
# Initialize dataframe with proper columns and columns typt
italy = similar(dataset,0)


Row,#EventID,Time,Latitude,Longitude,Depth/Km,Author,Catalog,Contributor,ContributorID,MagType,Magnitude,MagAuthor,EventLocationName,EventType
,Int64,DateTime,Float64,Float64,Float64,String,Missing,Missing,Missing,String,Float64,String,String,String


In [60]:
# year = 1985
# month = 1
# url = "http://webservices.ingv.it/fdsnws/event/1/query?starttime=" * string(year) * startDate[month] * "T00%3A00%3A00&endtime=" * string(year) * endDate[month] * "T23%3A59%3A59&minversion=100&orderby=time-asc&timezone=UTC&format=text&limit=15000"
# # put directly into dataframe. Note the delimiter:"|" and forcing the String to String
# dataset = CSV.read(download(url), DataFrame, delim="|", stringtype=String, 
#         select=[:Time, :Latitude, :Longitude, Symbol("Depth/Km"), :MagType, :Magnitude, :EventType], 
#         types=Dict(:Time=>String, :Latitude=>Float64, :Longitude=>Float64, Symbol("Depth/Km")=>Float64, :MagType=>String, :Magnitude=>Float64, :EventType=>String)) 
# # Initialize dataframe with proper columns and columns typt
# italy = similar(dataset,0)

Row,Time,Latitude,Longitude,Depth/Km,MagType,Magnitude,EventType
,String,Float64,Float64,Float64,String,Float64,String


In [91]:
year = 1985
month = 1
url = "http://webservices.ingv.it/fdsnws/event/1/query?starttime=" * string(year) * startDate[month] * "T00%3A00%3A00&endtime=" * string(year) * endDate[month] * "T23%3A59%3A59&minversion=100&orderby=time-asc&timezone=UTC&format=text&limit=15000"
# put directly into dataframe. Note the delimiter:"|" and forcing the String to String
dataset = CSV.read(download(url), DataFrame, delim="|", stringtype=String, 
        select=[:Time, :Latitude, :Longitude, Symbol("Depth/Km"), :MagType, :Magnitude, :EventType], 
        types=Dict(:Time=>String))#, :Latitude=>Float64, :Longitude=>Float64, :MagType=>String, :Magnitude=>Float64, :EventType=>String)) 
# Initialize dataframe with proper columns and columns typt
italy = similar(dataset,0)

Row,Time,Latitude,Longitude,Depth/Km,MagType,Magnitude,EventType
,String,Float64,Float64,Float64,String,Float64,String


In [ ]:
# Parse through years
for year=1985:2023
    # Parse through months
    for month=1:12
        filename = string(year) * "-" * string(month)
        filepath = "./output/" * filename
        url = "http://webservices.ingv.it/fdsnws/event/1/query?starttime=" * string(year) * startDate[month] * "T00%3A00%3A00&endtime=" * string(year) * endDate[month] * "T23%3A59%3A59&minversion=100&orderby=time-asc&timezone=UTC&format=text&limit=15000"
        
        # download(url,filepath)
        df = CSV.read(download(url), DataFrame, delim="|", stringtype=String,
                select=[:Time, :Latitude, :Longitude, Symbol("Depth/Km"), :MagType, :Magnitude, :EventType], 
                types=Dict(:Time=>String), #, :Latitude=>Float64, :Longitude=>Float64, Symbol("Depth/Km")=>Float64, :MagType=>String, :Magnitude=>Float64, :EventType=>String)) 
                validate=false)
        append!(italy,df);
    end
end

In [ ]:
italy.Time = chop.(italy.Time, tail=3)
italy.Time = DateTime.(italy.Time)

italy = italy[italy.EventType .== "earthquake", :]

In [93]:
rename!(italy,:Time => :Datetime, Symbol("Depth/Km") => :Depth, :MagType => :Magnitude_Type, :EventType => :Event_Type )

Row,Datetime,Latitude,Longitude,Depth,Magnitude_Type,Magnitude,Event_Type
,DateTime,Float64,Float64,Float64,String,Float64,String
1,1985-01-02T18:39:30.740,48.092,7.061,10.0,M,4.0,earthquake
2,1985-01-02T22:57:43.090,41.715,13.941,5.8,Md,2.6,earthquake
3,1985-01-03T00:46:16.990,41.677,13.931,6.1,Md,2.5,earthquake
4,1985-01-04T16:57:40.240,46.158,7.174,10.0,M,3.8,earthquake
5,1985-01-04T23:59:54.820,45.338,6.641,5.0,M,3.7,earthquake
6,1985-01-05T03:47:03.640,45.409,6.627,10.0,M,3.2,earthquake
7,1985-01-05T08:32:41.900,43.056,13.139,2.8,Md,2.6,earthquake
8,1985-01-05T10:13:26.800,45.15,7.86,20.8,Md,2.2,earthquake
9,1985-01-05T13:06:33.110,42.62,22.312,10.0,M,4.5,earthquake


In [80]:
CSV.write("../../data/italy.csv", italy)

"../../data/italy.csv"